# NTDS assignment 3: spectral graph theory
[Michaël Defferrard](http://deff.ch), *PhD student*, [EPFL](http://epfl.ch) [LTS2](http://lts2.epfl.ch)

The first two assignments were designed to warm you up. This third assignment is closer to what you'll have to do for the projects. It only misses the exploratory data analysis part (we'll do that later as an exercise). As such, this exercises is composed of two parts:
1. Data collection,
2. Data exploitation.

Our plan for the assignment:
1. Data collection: use a web API to collect the musical genre of 2000 songs.
2. Feature extraction: compute features from audio tracks.
3. Graph construction: construct a nearest-neighbor graph from the features.
4. Eigendecomposition: factorization of the graph Laplacian (c.f. spectral graph theory).
5. Visualization & Clustering: using the graph and eigenvectors to visualize the dataset and cluster the tracks.

In [ ]:
%matplotlib inline

import configparser
import os

import requests
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse, stats, spatial
import scipy.sparse.linalg
from sklearn import preprocessing, decomposition
import librosa
import IPython.display as ipd

plt.rcParams['figure.figsize'] = (17, 5)

If the above cell fails, it's most probably because you miss a package. Install it with e.g. `conda install librosa` or `pip install librosa`.

## 1 Data collection

Like in any data project, the first part of the assignment is to collect some data. 

### 1.1 Get the genre of a single track

As often, we need an API key for certain operations. Add the following to your `credentials.ini` file. I gave a key during the lab on November 6. If you were not there, ask one of your classmates.
```
[freemusicarchive]
api_key = MY-KEY
```

In [ ]:
# Read the confidential api key.
credentials = configparser.ConfigParser()
credentials.read(os.path.join('..', 'credentials.ini'))
api_key = credentials.get('freemusicarchive', 'api_key')

Your first task is to develop a function to retrieve the genre ID of a track given its track ID using the [FMA API](https://freemusicarchive.org/api).

Hints:
* A track might have multiple genres associated to it. Always return the first one and discard the others.
    * Note: you should never discard data blindly. I selected the tracks so that this is not a problem.
* The `get_genre` function takes an integer as input, the track ID, and returns another integer, the genre ID.

In [ ]:
def get_genre(track_id):
    """Returns the genre of a track by querying the API."""
    BASE_URL = 'https://freemusicarchive.org/api/get/tracks.json'
    url = '{}?track_id={}&api_key={}'.format(BASE_URL, track_id, api_key)
    response = requests.get(url).json()
    return int(response['dataset'][0]['track_genres'][0]['genre_id'])

# A correct implementation should pass the below test.
assert get_genre(1219) == 38

### 1.2 Create a table of tracks

The `fma_tracks.csv` file contains a list of 2'000 track IDs that we will use through this assignment.

In [ ]:
tracks = pd.read_csv('../data/fma_tracks.csv', index_col=0)
print('You got {} track IDs.'.format(len(tracks)))

Once imported by pandas, each row of the DataFrame represents a track.

In [ ]:
tracks.head(4)

### 1.3 Add the genre to the table

Your task is to add a `genre` column to the above created `tracks` DataFrame. The column should contain an integer which represents the genre of the track, i.e. the return value of the `get_genre` function you developed.

Hints:
* When developing, retrieve the genre of the first 10 tracks only. Only once your code works run it through all the tracks. That will save you time.
* As we want to apply one function (`get_genre`) to many data samples (the 2000 track IDs), you may want to use a functional approach. Check out `tracks.apply()` or the built-in `map`. In Python, you can declare an [anonymous function](https://en.wikipedia.org/wiki/Anonymous_function) as `lambda x: x + 1`.
* Your table should look like the below table, except with the correct number instead of 0.

In [ ]:
tracks['genre'] = 0
tracks.head(4)

In [ ]:
tracks = tracks[:10]

#tracks['genre'] = tracks.apply(lambda track: get_genre(track.name), axis=1)

# Alternatively:
# tracks['genre'] = list(map(get_genre, tracks.index))

# Alternatively:
# for tid in tqdm(tracks.index[:10]):
#     tracks.at[tid, 'genre'] = get_genre(tid)

In [ ]:
tracks.head(4)

### 1.4 Save the data

To avoid having to collect the data everytime you restart the IPython kernel, save the DataFrame as a CSV file.

In [ ]:
#tracks.to_csv('../data/fma_tracks_with_genre.csv')

You can now load it back with the following call instead of running the code in sections 1.1 to 1.3.

In [ ]:
tracks = pd.read_csv('../data/fma_tracks_with_genre.csv', index_col=0)

### 1.5 Data cleaning

As always, data cleaning is necessary when dealing with real (as opposed to synthetic) data. In this case, we only need to "summarize the genres". The tracks I've selected for the assignment belong to either one of the following *top-level genres*: Rock (`genre_id=12`) and Hip-Hop (`genre_id=21`). There *actual genre(s)* might however be more specific and be a sub-genre of those. For example Punk is a sub-genre of Rock. You can explore the genre hierarchy on the [Free Music Archive](http://freemusicarchive.org/genre/Rock/). The below function will return the correct top-level genre for any of the sub-genres you'll encounter.

In [ ]:
def get_top_genre(genre_id):
    return 21 if genre_id in [21, 83, 100, 539, 542, 811] else 12

tracks = tracks.applymap(lambda genre: get_top_genre(genre))
tracks.head(4)

If everything went fine, you should now have 1000 Rock (`genre_id=12`) and 1000 Hip-Hop (`genre_id=12`) tracks.

In [ ]:
tracks['genre'].value_counts()

## 2 Feature extraction

As is often the case, the data at hand is too large to be dealt with directly. We have to represent it with a smaller set of features, chosen to be maximally relevant to the task. (Manual feature extraction can sometimes be replaced by end-to-end learning systems.)

### 2.1 Get raw data

Decompress the zip you found on Moodle and write below the path to the mp3 files. The zip contains a 30s excerpt for each track listed in the `tracks` table. The name of the audio file is simply the track ID.

But first, let's listen to some music.

In [ ]:
def get_path(track_id):
    return os.path.join('..', 'data', '{:06d}.mp3'.format(track_id))

# 1. Get the path to the file.
filepath = get_path(tracks.index[0])
print('File: {}'.format(filepath))

# 2. Decode the mp3 and load the audio in memory.
audio, sampling_rate = librosa.load(filepath, sr=None, mono=True)
print('Duration: {:.2f}s, {} samples'.format(audio.shape[-1] / sampling_rate, audio.size))

# 3. Load the audio in the browser and play it.
start, end = 7, 17
ipd.Audio(data=audio[start*sampling_rate:end*sampling_rate], rate=sampling_rate)

### 2.2 Spectral features

For music, the [mel-frequency cepstral coefficients (MFCCs)](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum) are often relevant spectral features. The parameter here (except the choice of feature), is the number of coefficients to compute.

Hint:
* Use the function `librosa.feature.mfcc` to compute those features.

In [ ]:
N_MFCC = 20

def compute_mfcc(track_id):
    audio, sampling_rate = librosa.load(get_path(track_id), sr=None, mono=True)
    return librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=N_MFCC)

assert compute_mfcc(tracks.index[0]).shape == (N_MFCC, 2582)

### 2.3 Summary statistics

The `compute_mfcc` function we developed above computes `N_MFCC` coefficients per window in time. Notice that we computed `N_MFCC` x 2582 coefficients for the first track. To have a fixed representation for each complete track (and not for each window of 2048 samples), we need to aggregate those coefficients along time. We'll do it with 7 summary statistics: the minimum, the maximum, the median, and the first 4 moments, i.e. the mean, the standard deviation, the skew and the kurtosis.

Below, we construct the DataFrame that will hold our features. Note the use of a hierarchical index. Because I'm a nice guy, I computed the features for most tracks.

In [ ]:
features = pd.read_csv('../data/fma_features.csv', index_col=0, header=[0, 1, 2])
assert (tracks.index == features.index).all()

features.tail(4)

Though I forgot to compute them for the first 10 tracks. ;-) Complete the below code to do that.

Hints:
* Functions to compute the mentioned statistics are found in `np` and `stats` (from `scipy`, see imports).
* We use `tqdm` to show progress on long computations. For example: `for i in tqdm(range(10)): print(i)`.

In [ ]:
for tid in tqdm(tracks.index[:10]):
    mfcc = compute_mfcc(tid)
    features.at[tid, ('mfcc', 'mean')] = np.mean(mfcc, axis=1)
    features.at[tid, ('mfcc', 'std')] = np.std(mfcc, axis=1)
    features.at[tid, ('mfcc', 'skew')] = stats.skew(mfcc, axis=1)
    features.at[tid, ('mfcc', 'kurtosis')] = stats.kurtosis(mfcc, axis=1)
    features.at[tid, ('mfcc', 'median')] = np.median(mfcc, axis=1)
    features.at[tid, ('mfcc', 'min')] = np.min(mfcc, axis=1)
    features.at[tid, ('mfcc', 'max')] = np.max(mfcc, axis=1)

features['mfcc','mean'].head(4)

### 2.4 Feature selection

Once you are done with the rest of the assignment and reach section 5.4, come back here and try to identify the most relevant features for our end goal, i.e. data visualization and clustering.

In [ ]:
features.drop(('mfcc', 'kurtosis'), axis=1, inplace=True)
features.drop(('mfcc', 'skew'), axis=1, inplace=True)

features.drop(('mfcc', 'min'), axis=1, inplace=True)
features.drop(('mfcc', 'max'), axis=1, inplace=True)

### 2.5 Feature normalization

Most algorithms expect (or work better) if the data is centered and standardized.

In [ ]:
features -= features.mean(axis=0)
features /= features.std(axis=0)

## 3 Graph construction

As opposed to social networks, the brain, or a road network, this dataset does not exhibit a natural graph. But we can always build one! In this case, we will build a similarity graph between tracks. In such a graph, each track is represented as a node and the weight of an edge will be an indication of how similar two tracks are (1 meaning identical, and 0, i.e. no edge, meaning very different). Such graphs are useful for e.g. recommendation. If 10 tracks you liked are strongly connected to an 11th one, you'll probably like that one too (if the similarity measure is relevant).

### 3.1 Compute distances

Metric

The Euclidean, or l2, distance is defined as $$d(i,j) = \|x_i - x_j\|_2$$

Hints:
* Save yourself some pain and use `pdist` and `squareform` from `scipy.spatial.distance`.

In [ ]:
distances = spatial.distance.pdist(features, metric='euclidean')
distances = spatial.distance.squareform(distances)

In [ ]:
plt.hist(distances.reshape(-1), bins=50);

Why are some distances equal to zero?

In [ ]:
print('{} distances equal exactly zero.'.format(np.sum(distances == 0)))

### 3.2 Compute the weight matrix

Gaussian kernel $$\mathbf{W}(i,j) = \exp \left( \frac{-d^2(i, j)}{\sigma^2} \right)$$

In [ ]:
kernel_width = distances.mean()
weights = np.exp(-distances**2 / kernel_width**2)

np.fill_diagonal(weights, 0)

What kind of graph is that? Fully connected.

Sparsify the graph. Either knn or $\epsilon$. knn better to enforce connectedness.

In [ ]:
fix, axes = plt.subplots(2, 2, figsize=(17, 8))
def plot(weights, axes):
    axes[0].spy(weights)
    axes[1].hist(weights[weights > 0].reshape(-1), bins=50);
plot(weights, axes[:, 0])

if False:
    epsilon = np.percentile(weights, 80)
    weights[weights < epsilon] = 0
else:
    NEIGHBORS = 10
    idx = np.argsort(weights)[:, :-NEIGHBORS]
    for i in range(weights.shape[0]):
        weights[i, idx[i, :]] = 0
    weights = np.maximum(weights, weights.T)

plot(weights, axes[:, 1])

### 3.3 Compute the Laplacian

In [ ]:
degrees = weights.sum(0)

plt.hist(degrees, bins=50);

Shall we use the un-normalized or normalized Laplacian? Choose and justify.

In [ ]:
# Combinatorial Laplacian.
laplacian = np.diag(degrees) - weights

# Normalized Laplacian.
deg_inv = np.diag(1 / np.sqrt(degrees))
laplacian = deg_inv @ laplacian @ deg_inv

# Alternatively:
# laplacian = np.identity(weights.shape[0]) - deg_inv @ weights @ deg_inv

plt.spy(laplacian)

In [ ]:
laplacian = sparse.csr_matrix(laplacian)

How many edges?

In [ ]:
print('{} edges out of {} x {} = {}'.format(laplacian.nnz, *weights.shape, weights.size))

### 3.4 Bonus

Can you think of a way to observe if the two genres form clusters in the graph we created?

Hint: Use only the weight matrix / laplacian and the labels.

Sort the rows and columns given the labels.

## 4 Eigendecomposition of the graph Laplacian

No need to compute the Fourier basis, only the Fiedler vector, i.e. the eigenvector associated to $\lambda_2$.

Use one of the following functions: `np.linalg.eig`, `np.linalg.eigh`, `sparse.linalg.eigs`, `sparse.linalg.eigsh`. Justify your choice.

In [ ]:
eigenvalues, eigenvectors = sparse.linalg.eigsh(laplacian, k=10, which='SM')

# That's much slower:
# eigenvalues, eigenvectors = np.linalg.eigh(laplacian.toarray())

In [ ]:
plt.plot(eigenvalues, '.-', markersize=15);

### 4.1 Connectedness

Is the graph connected? Justify. Knowing how we built the graph, can we ensure it is connected?

In [ ]:
print(eigenvalues)

**Your answer here.** The graph is connected because only one eigenvalue equals zero (multiplicity one). The graph is connected by construction because of kNN.

### 4.2 Eigenvector question

What do you expect as the result of the below computation? Justify. Do you get the value you expected? If not, why?

Note that `x @ y` (introduced in Python 3.5) is equivalent to `np.matmul(x, y)`. You should prefer the former as it makes it easier to read formulas.

In [ ]:
np.sum(laplacian @ (2 * eigenvectors[:, 0]))

**Your answer here.** We expect zero because the first eigenvalue is zero. As such, the first eigenvector lies in the null-space of the Laplacian. The small error is due to numerical precision.

## 5 Visualization and clustering

Finally, let's use the data and graph we prepared. When [exploring data](https://en.wikipedia.org/wiki/Exploratory_data_analysis), it's often useful to visualize an entire dataset. Because for us humans it's hard to look at data in 140 dimensions, we need to somehow reduce the dimensionality to 2 or 3 and visualize the data in this more familiar space. While such a reduction will obviously be destructive, many algorithms have been developed to preserve certain properties.

### 5.1 Principal component analysis (PCA)

[PCA](https://en.wikipedia.org/wiki/Principal_component_analysis) is a standard algorithm to reduce the dimensionality of a dataset. It computes the axes of principal variance and project the data on them. It does not use any graph. We show it here for comparison.

In [ ]:
features_pca = decomposition.PCA(n_components=2).fit_transform(features)
genres = preprocessing.LabelEncoder().fit_transform(tracks['genre'])
plt.scatter(features_pca[:,0], features_pca[:,1], c=genres, cmap='RdBu', alpha=0.5);

### 5.2 Graph embedding

Note how this plot summarizes well 2GB of data and 2000 tracks.

In [ ]:
plt.scatter(eigenvectors[:, 1], eigenvectors[:, 2], c=genres, cmap='RdBu', alpha=0.5);

### 5.3 Clustering

Note how we did not try to build a machine to recognize the musical genre given a track (that would have been a [classification problem](https://en.wikipedia.org/wiki/Statistical_classification)). We did merely try to visualize the data, by means of PCA and a graph embedding algorithm. What does it tell us that genre clearly appears in our visualization?

**Your answer here.** Genres are useful to categorize songs and organize music collections. Or, the chosen features are (overly?) sensitive to musical genre. Beware of the pitfalls!

As such, we can try to cluster the tracks with the Fiedler vector, and look if the (unsupervised) clustering agrees with the *ground truth* genre categorization.

In [ ]:
labels = (eigenvectors[:, 1] > 0)

plt.scatter(eigenvectors[:, 1], eigenvectors[:, 2], c=labels, cmap='RdBu', alpha=0.5);

### 5.4 Error rate

How many tracks were wrongly identified?

In [ ]:
err = np.sum(np.abs(labels - genres))
err = err if err < len(labels)/2 else len(labels) - err
print('{} errors ({:.2%})'.format(err, err/len(labels)))

Tune some parameters (e.g. `kernel_width`, `NEIGHBORS`) and discard some features (see section 2.4) to get less errors. You should get an error rate below 15% (i.e. less than 300 errors in total). Try to understand the effect of each parameter. Be aware that tuning the parameters on a specific dataset will lead to [overfitting](https://en.wikipedia.org/wiki/Overfitting).

## 6 Conclusion

Among other things, this assignment showed us that a graph can be useful for e.g. visualization or clustering, even when there is none in the original data. We exercised here two steps of the Data Science process: i) data collection, and ii) data exploitation. The exploitation of the data showed us that a machine can discern musical genres by looking at pairwise distances between spectral features extracted from audio recordings.

### 6.1 Bonus

What is the name of the technique we used to visualize the data in the last two plots? What does it try to preserve when reducing the dimensionality (of the ambiant space) from 140 to 2?

**Your answer here.** The technique is named Laplacian eigenmaps and has been introduced by Belkin and Niyogi in their paper [Laplacian Eigenmaps for Dimensionality Reduction and Data](http://web.cse.ohio-state.edu/~belkin.8/papers/LEM_NC_03.pdf). The algorithm tries to preserve the distance between data samples.